# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [274]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [275]:
y = x + x

In [276]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [277]:
import syft as sy

In [278]:
# syft is an extension of torch, and we can create a hook, which modified pytrch with new functionalities
hook = sy.TorchHook(th)

W0623 14:35:03.715316 26688 hook.py:97] Torch was already hooked... skipping hooking process


In [279]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [280]:
# here we create a worker called bob
bob = sy.VirtualWorker(hook, id="bob")

In [281]:
#bob has no objects
bob._objects

{94921619732: tensor([ 0.,  1., 23.,  3.,  1.,  3.,  3.]),
 61691997068: tensor([ 0.,  1., 23.,  3.,  1.,  3.,  3.]),
 11385318184: tensor([1, 2, 3, 4, 5]),
 45241903796: tensor([[0.],
         [0.],
         [0.],
         [0.]], grad_fn=<MmBackward>),
 99444241652: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True),
 86603234360: tensor([1, 2, 3, 4, 5]),
 68742634147: tensor([1, 2, 3, 4, 5]),
 96677417421: tensor([1, 1, 1, 1, 1]),
 79140053018: tensor(20., grad_fn=<SumBackward0>),
 75463042275: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True),
 26691665293: tensor([[0.0536],
         [0.9463]], requires_grad=True),
 26351277307: tensor([[0.9997],
         [0.9328],
         [0.0670],
         [0.0000]], grad_fn=<MmBackward>),
 83584553351: tensor([1, 2, 3, 4, 5]),
 71505585853: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 70719746265: tensor([[1.],
         [1.]], requires_grad=

In [282]:
# we create an oobject x
x = th.tensor([1,2,3,4,5])

In [283]:
# we send the bject x to bob, and store whatever is returned
x = x.send(bob)
# what is returned from this send function is a pointer to the remote object
print("value of pointer to the remote object", x)
# a pointer here is a tensor, and thus has the full tensor API at its disposal
# however, each command is serialised in JSON/tuple format sent to bob and bob executes the commands on our behalf
# and bob returns to us,a pointer to the new object

value of pointer to the remote object (Wrapper)>[PointerTensor | me:73536145961 -> bob:38678573049]


In [284]:
bob._objects

{94921619732: tensor([ 0.,  1., 23.,  3.,  1.,  3.,  3.]),
 61691997068: tensor([ 0.,  1., 23.,  3.,  1.,  3.,  3.]),
 11385318184: tensor([1, 2, 3, 4, 5]),
 45241903796: tensor([[0.],
         [0.],
         [0.],
         [0.]], grad_fn=<MmBackward>),
 99444241652: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True),
 86603234360: tensor([1, 2, 3, 4, 5]),
 68742634147: tensor([1, 2, 3, 4, 5]),
 96677417421: tensor([1, 1, 1, 1, 1]),
 79140053018: tensor(20., grad_fn=<SumBackward0>),
 75463042275: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True),
 26691665293: tensor([[0.0536],
         [0.9463]], requires_grad=True),
 26351277307: tensor([[0.9997],
         [0.9328],
         [0.0670],
         [0.0000]], grad_fn=<MmBackward>),
 83584553351: tensor([1, 2, 3, 4, 5]),
 71505585853: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 70719746265: tensor([[1.],
         [1.]], requires_grad=

In [285]:
x.location == bob

True

In [286]:
x.id_at_location

38678573049

In [287]:
x.id
# when we try to perform a command using x, its going to send a meesga to self.location and say
# hey bob, find the tensor with this ID and execute the command I want you to execute

73536145961

In [288]:
# all tensor is pysyft has an owner, since we have a client, owner default to be me.
# this reveals another worker that was created when we first created and hooked pytorch in pysyft
# this is called local_worker
x.owner

# what we really do when executing a command WRT x, we say, hey, local_worker, 
# contact bob and tellh im to do this!

<VirtualWorker id:me #objects:0>

In [289]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [290]:
x

(Wrapper)>[PointerTensor | me:73536145961 -> bob:38678573049]

In [291]:
# get is like, read etc, but it actually sends the data from bob to us, and bob no longer 
# carrying info on that tensor
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [292]:
bob._objects

{94921619732: tensor([ 0.,  1., 23.,  3.,  1.,  3.,  3.]),
 61691997068: tensor([ 0.,  1., 23.,  3.,  1.,  3.,  3.]),
 11385318184: tensor([1, 2, 3, 4, 5]),
 45241903796: tensor([[0.],
         [0.],
         [0.],
         [0.]], grad_fn=<MmBackward>),
 99444241652: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True),
 86603234360: tensor([1, 2, 3, 4, 5]),
 68742634147: tensor([1, 2, 3, 4, 5]),
 96677417421: tensor([1, 1, 1, 1, 1]),
 79140053018: tensor(20., grad_fn=<SumBackward0>),
 75463042275: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True),
 26691665293: tensor([[0.0536],
         [0.9463]], requires_grad=True),
 26351277307: tensor([[0.9997],
         [0.9328],
         [0.0670],
         [0.0000]], grad_fn=<MmBackward>),
 83584553351: tensor([1, 2, 3, 4, 5]),
 71505585853: tensor([[1., 1.],
         [0., 1.]], requires_grad=True),
 70719746265: tensor([[1.],
         [1.]], requires_grad=

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [293]:
# try this project here!
# imports
import torch as th
import syft as sy

In [294]:
# syft is an extension of torch, and we can create a hook, which modified pytrch with new functionalities
# hook pytorch to pysyft
hook = sy.TorchHook(th)

W0623 14:35:07.946596 26688 hook.py:97] Torch was already hooked... skipping hooking process


In [295]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [296]:
x = th.Tensor([0,1,23,3,1,3,3])

In [297]:
x_ptr = x.send(bob, alice)

In [298]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:55711471073 -> bob:30801514335]
	-> (Wrapper)>[PointerTensor | me:40937031974 -> alice:96984699511]

In [299]:
# a = x_ptr.get()
# print(a)

In [300]:
x_ptr.get(sum_results=True)

tensor([ 0.,  2., 46.,  6.,  2.,  6.,  6.])

# Lesson: Introducing Remote Arithmetic

In [301]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [302]:
x

(Wrapper)>[PointerTensor | me:11837244052 -> bob:76161182863]

In [303]:
y

(Wrapper)>[PointerTensor | me:12715246602 -> bob:70984203469]

In [304]:
z = x + y

In [305]:
z

(Wrapper)>[PointerTensor | me:93830579793 -> bob:28055667220]

In [306]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [307]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:63375865604 -> bob:59238880103]

In [308]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [309]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [310]:
z = (x + y).sum()

In [311]:
z.backward()

(Wrapper)>[PointerTensor | me:24355474684 -> bob:81628450934]

In [312]:
x = x.get()

In [313]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [314]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [315]:
import torch
input = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1], [0], [0]]).send(bob)

In [316]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [317]:
for i in range(10):
    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [318]:
bob = sy.VirtualWorker(hook, id="bob")

bob = bob.clear_objects()

In [319]:
bob._objects

{}

In [320]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [321]:
bob._objects

{}

In [322]:
del x

In [323]:
bob._objects

{}

In [324]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [325]:
bob._objects

{}

In [326]:
x = "asdf"

In [327]:
bob._objects

{}

In [328]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [329]:
x

(Wrapper)>[PointerTensor | me:22357143700 -> bob:7623945081]

In [330]:
bob._objects

{}

In [331]:
x = "asdf"

In [332]:
bob._objects

{}

In [333]:
del x

In [334]:
bob._objects

{}

In [335]:
bob = bob.clear_objects()
bob._objects

{}

In [336]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [337]:
bob._objects
# x = th.tensor([1,2,3,4,5]).send(bob)

{}

In [338]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [339]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:27093792558 -> bob:28281769319]
Tensor B: tensor([1, 1, 1, 1, 1])

In [340]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [341]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:0> while the other is on <VirtualWorker id:alice #objects:5>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [419]:
# try this project here!
# imports
import torch as th
import syft as sy
from torch import nn, optim

In [420]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

W0623 14:59:51.385064 26688 hook.py:97] Torch was already hooked... skipping hooking process


In [421]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [422]:
data = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [423]:
# a toy model
model = nn.Linear(2, 1)

In [424]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [425]:
def train(iters=20):
    for i in range(iters):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)

train()

tensor(3.6667)
tensor(0.9569)
tensor(0.5119)
tensor(0.3232)
tensor(0.2089)
tensor(0.1357)
tensor(0.0884)
tensor(0.0578)
tensor(0.0379)
tensor(0.0249)
tensor(0.0165)
tensor(0.0110)
tensor(0.0073)
tensor(0.0049)
tensor(0.0033)
tensor(0.0023)
tensor(0.0016)
tensor(0.0011)
tensor(0.0008)
tensor(0.0005)


In [426]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [427]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [428]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [429]:
model = nn.Linear(2, 1)
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [435]:
def train(iterations=20):
    model = nn.Linear(2, 1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(iterations):
        for _data, _target in datasets:

            # send model to remote worker/data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model=model.get()

            print(loss.get())

In [436]:
train()

tensor(1.0617, requires_grad=True)
tensor(2.5703, requires_grad=True)
tensor(0.8727, requires_grad=True)
tensor(1.5354, requires_grad=True)
tensor(0.5297, requires_grad=True)
tensor(0.9379, requires_grad=True)
tensor(0.3180, requires_grad=True)
tensor(0.5784, requires_grad=True)
tensor(0.1928, requires_grad=True)
tensor(0.3601, requires_grad=True)
tensor(0.1184, requires_grad=True)
tensor(0.2266, requires_grad=True)
tensor(0.0738, requires_grad=True)
tensor(0.1442, requires_grad=True)
tensor(0.0468, requires_grad=True)
tensor(0.0928, requires_grad=True)
tensor(0.0303, requires_grad=True)
tensor(0.0605, requires_grad=True)
tensor(0.0200, requires_grad=True)
tensor(0.0400, requires_grad=True)
tensor(0.0134, requires_grad=True)
tensor(0.0267, requires_grad=True)
tensor(0.0092, requires_grad=True)
tensor(0.0181, requires_grad=True)
tensor(0.0064, requires_grad=True)
tensor(0.0124, requires_grad=True)
tensor(0.0045, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0032, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [119]:
# try this project here!
# imports
import torch as th
import syft as sy

# syft is an extension of torch, and we can create a hook, which modified pytrch with new functionalities
# hook pytorch to pysyft
hook = sy.TorchHook(th)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

W0628 17:11:42.776902 18336 hook.py:97] Torch was already hooked... skipping hooking process


In [120]:
bob.clear_objects

<bound method BaseWorker.clear_objects of <VirtualWorker id:bob #objects:0>>

In [121]:
alice.clear_objects

<bound method BaseWorker.clear_objects of <VirtualWorker id:alice #objects:0>>

In [122]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [123]:
x = x.send(alice)

In [124]:
bob._objects

{6821494378: tensor([1, 2, 3, 4, 5])}

In [125]:
# on alices machine, alice has a ptr from alice to bob, and our ptr no 
# longer points to bob, it points to alice, 
# when we send a message to contact this tensor, ie add tesnros, it would 
# first go to alice, and get processed there, and then forward the message
# to bobs machine and process it there
alice._objects

{50534874292: (Wrapper)>[PointerTensor | alice:50534874292 -> bob:6821494378]}

In [15]:
y = x+x

In [16]:
# now we have a new ptr, still pointing to alices machine.
y

(Wrapper)>[PointerTensor | me:94195136172 -> alice:96426943998]

In [17]:
# but if we look at bobs machine, we will see bob has 2 tensors, x and y
bob._objects

{60928308416: tensor([1, 2, 3, 4, 5]),
 73019995941: tensor([ 2,  4,  6,  8, 10])}

In [18]:
# alice ahs two tensors, but theyre pointers! 
# and y has the same dependancy chain as x
alice._objects
# if we have some other machine say; Jim, come along and say - Hey, alice, 
# tell bob I say hey - alice will refuse, and if jim said, hey bob
# take out money from yours and alices joint account, bob will say no
# since its jointly owned by bob and alice

{88294058088: (Wrapper)>[PointerTensor | alice:88294058088 -> bob:60928308416],
 96426943998: (Wrapper)>[PointerTensor | alice:96426943998 -> bob:73019995941]}

In [ ]:
# if we did opertations between tensros with different tensor chains we 
# will get an error

In [19]:
jon = sy.VirtualWorker(hook, id='jon')

In [31]:
bob.clear_objects()
alice.clear_objects()
jon.clear_objects()

<VirtualWorker id:jon #objects:0>

In [32]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(alice)
# y = th.tensor([1,2,3,4,5]).send(bob).send(jon) # this will break if run

In [33]:
z = x +y

# data is on bobs mnachine, and since jon and alice dony agree, the chain 
# is broken! 

# You tried to call __add__ involving two tensors which are not on the same 
# machine! One tensor is on <VirtualWorker id:alice #objects:2> while the 
# other is on <VirtualWorker id:jon #objects:1>. Use a combination of
# .move(), .get(), and/or .send() to co-locate them to the same machine.

In [42]:
bob._objects

{52564484384: tensor([ 2,  4,  6,  8, 10])}

In [43]:
alice._objects

{70012606394: (Wrapper)>[PointerTensor | alice:70012606394 -> bob:52564484384]}

In [118]:
x = x.get()
x
# bob now has x, but alice doesnt since she sent it to us! 
# and we point directly to bobs data
# run it again, and its removed from bob too! 

NameError: name 'x' is not defined

In [41]:
del y

# Lesson: Pointer Chain Operations

In [46]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [48]:
alice._objects
bob._objects

{}

In [52]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [53]:
bob._objects

{23838917203: tensor([1, 2, 3, 4, 5])}

In [54]:
alice._objects

{3810586069: (Wrapper)>[PointerTensor | alice:3810586069 -> bob:23838917203]}

In [57]:
x.remote_get()

(Wrapper)>[PointerTensor | me:80743642792 -> alice:3810586069]

In [74]:
bob._objects

{80743642792: tensor([1, 2, 3, 4, 5])}

In [81]:
alice._objects

{38377260982: tensor([1, 2, 3, 4, 5, 6])}

In [73]:
x.move(bob)

(Wrapper)>[PointerTensor | me:80743642792 -> bob:80743642792]

In [79]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [82]:
# z = th.tensor([1,2,3,4,5,6]).move(alice)

# Final Project for Federared Learning:

1. Here, we want to do FL where the central server is not trusted. 
2. The grads should not come to the central server in raw format - we should use the .move command to move the gradients to one worker, sum them there, and then bring the btach to the central server - meaning they never see the raw gradients for the models 

The data here will be changed so its not longer using MNIST - since I cant figure out how to send the images to each worker - only work around would be to have each user read in X images etc from the data. 

The next approach to try is to set up some hard coded data objects - maybe say 3 workers, 2 learners and 1 central trusted worker - and the data will be similar to those above [[1, 0] ... etc]



In [119]:
#imports

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torch as th
import syft as sy

from torch import nn, optim
from torch.utils.data import SubsetRandomSampler
from torchvision import datasets, transforms

try:
    from syft.frameworks.torch.differential_privacy import pate
except ImportError:
    print("ERROR: syft not installed - Install syft and trying import again")
    !pip install syft
    from syft.frameworks.torch.differential_privacy import pate

import torch

In [120]:
hook = sy.TorchHook(th)

worker_1 = sy.VirtualWorker(hook, id="w1")
worker_2 = sy.VirtualWorker(hook, id="w2")
worker_3 = sy.VirtualWorker(hook, id="w3")
worker_4 = sy.VirtualWorker(hook, id="w4")
worker_5 = sy.VirtualWorker(hook, id="w5")

W0629 16:06:58.204363 22028 hook.py:97] Torch was already hooked... skipping hooking process


In [121]:
worker_1.add_workers([worker_2, worker_3, worker_4])
worker_2.add_workers([worker_1, worker_3, worker_4])
worker_3.add_workers([worker_2, worker_1, worker_4])
worker_4.add_workers([worker_2, worker_3, worker_1])

W0629 16:06:58.371696 22028 base.py:628] Worker w2 already exists. Replacing old worker which could cause                     unexpected behavior
W0629 16:06:58.372655 22028 base.py:628] Worker w3 already exists. Replacing old worker which could cause                     unexpected behavior
W0629 16:06:58.373151 22028 base.py:628] Worker w4 already exists. Replacing old worker which could cause                     unexpected behavior
W0629 16:06:58.373647 22028 base.py:628] Worker w1 already exists. Replacing old worker which could cause                     unexpected behavior
W0629 16:06:58.374142 22028 base.py:628] Worker w3 already exists. Replacing old worker which could cause                     unexpected behavior
W0629 16:06:58.375135 22028 base.py:628] Worker w4 already exists. Replacing old worker which could cause                     unexpected behavior
W0629 16:06:58.376127 22028 base.py:628] Worker w2 already exists. Replacing old worker which could cause                   

<VirtualWorker id:w4 #objects:8>

In [122]:
input = th.tensor([[0., 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], 
                   [1., 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]], requires_grad=True)

In [123]:
target = th.tensor([[0.],[1],[1],[1], 
                    [1.],[1],[1],[1]], requires_grad=True)

In [124]:
w1_data = input[0:2].send(worker_1)
w2_data = input[2:4].send(worker_2)
w3_data = input[4:6].send(worker_3)
w4_data = input[6:8].send(worker_4)

In [125]:
w1_target = target[0:2].send(worker_1)
w2_target = target[2:4].send(worker_2)
w3_target = target[4:6].send(worker_3)
w4_target = target[6:8].send(worker_4)

In [126]:
print("worker 1: ", worker_1._objects)
print("worker 2: ", worker_2._objects)
print("worker 3: ", worker_3._objects)
print("worker 4: ", worker_4._objects)

worker 1:  {12373475095: Parameter containing:
tensor([[-0.0438,  0.3927, -1.4824]], requires_grad=True), 82847667062: Parameter containing:
tensor([-4.4506], requires_grad=True), 47428007536: tensor([[0., 0., 0.],
        [0., 0., 1.]], requires_grad=True), 66392323362: tensor([[0.],
        [1.]], requires_grad=True), 13304156053: Parameter containing:
tensor([[ 0.2107, -0.5582, -2.1015]], requires_grad=True), 59978876911: Parameter containing:
tensor([-3.6290], requires_grad=True), 86087564980: tensor([[0.4023],
        [0.8359]], grad_fn=<AddmmBackward>), 15823817582: tensor([[0., 0., 0.],
        [0., 0., 1.]], requires_grad=True), 77237561809: tensor([[0., 0., 0.],
        [0., 0., 1.]], requires_grad=True), 80585001764: tensor([[0.],
        [1.]], requires_grad=True)}
worker 2:  {56785032151: tensor([[0., 1., 0.],
        [0., 1., 1.]], requires_grad=True), 36793182926: tensor([[1.],
        [1.]], requires_grad=True), 70112119781: Parameter containing:
tensor([[ 0.2107, -4.558

In [127]:
model = nn.Linear(3, 1)

In [155]:
for j in range(10):
    w1_model = model.copy().send(worker_1)
    w2_model = model.copy().send(worker_2)
    w3_model = model.copy().send(worker_3)
    w4_model = model.copy().send(worker_4)

    w1_opt = optim.SGD(params=w1_model.parameters(), lr=0.1)
    w2_opt = optim.SGD(params=w2_model.parameters(), lr=0.1)
    w3_opt = optim.SGD(params=w3_model.parameters(), lr=0.1)
    w4_opt = optim.SGD(params=w4_model.parameters(), lr=0.1)
    for i in range(10):
        w1_opt.zero_grad()
        w1_pred = w1_model(w1_data)
        w1_loss = ((w1_pred - w1_target)**2).sum()
        w1_loss.backward()
        w1_opt.step()
        w1_loss = w1_loss.get().data

        w2_opt.zero_grad()
        w2_pred = w2_model(w2_data)
        w2_loss = ((w2_pred - w2_target)**2).sum()
        w2_loss.backward()
        w2_opt.step()
        w2_loss = w2_loss.get().data

        w3_opt.zero_grad()
        w3_pred = w3_model(w3_data)
        w3_loss = ((w3_pred - w3_target)**2).sum()
        w3_loss.backward()
        w3_opt.step()
        w3_loss = w3_loss.get().data

        w4_opt.zero_grad()
        w4_pred = w4_model(w4_data)
        w4_loss = ((w4_pred - w4_target)**2).sum()
        w4_loss.backward()
        w4_opt.step()
        w4_loss = w4_loss.get().data

    w1_model.move(worker_5)
    w2_model.move(worker_5)
    w3_model.move(worker_5)
    w4_model.move(worker_5)

    avg_weight = (w1_model.weight.data + w2_model.weight.data + w3_model.weight.data + w4_model.weight.data) / 4
    model.weight.data.copy_(avg_weight.get())

    avg_bias = (w1_model.bias.data + w2_model.bias.data + w3_model.bias.data + w4_model.bias.data) / 4
    model.bias.data.copy_(avg_bias.get())
    
    print("w1: " + str(w1_loss) + " w2: " + str(w2_loss))
    print("w3: " + str(w3_loss) + " w4: " + str(w4_loss))

w1: tensor(0.0599) w2: tensor(0.0059)
w3: tensor(0.0085) w4: tensor(0.0033)
w1: tensor(0.0595) w2: tensor(0.0061)
w3: tensor(0.0083) w4: tensor(0.0033)
w1: tensor(0.0593) w2: tensor(0.0062)
w3: tensor(0.0082) w4: tensor(0.0032)
w1: tensor(0.0590) w2: tensor(0.0063)
w3: tensor(0.0081) w4: tensor(0.0032)
w1: tensor(0.0588) w2: tensor(0.0064)
w3: tensor(0.0079) w4: tensor(0.0032)
w1: tensor(0.0585) w2: tensor(0.0065)
w3: tensor(0.0079) w4: tensor(0.0032)
w1: tensor(0.0584) w2: tensor(0.0066)
w3: tensor(0.0078) w4: tensor(0.0032)
w1: tensor(0.0582) w2: tensor(0.0067)
w3: tensor(0.0077) w4: tensor(0.0032)
w1: tensor(0.0580) w2: tensor(0.0067)
w3: tensor(0.0076) w4: tensor(0.0032)
w1: tensor(0.0579) w2: tensor(0.0068)
w3: tensor(0.0076) w4: tensor(0.0032)


In [117]:
worker_1.clear_objects()
worker_2.clear_objects()
worker_3.clear_objects()
worker_4.clear_objects()
worker_5.clear_objects()

<VirtualWorker id:w5 #objects:0>

In [118]:
print("worker 1: ", worker_1._objects)
print("worker 2: ", worker_2._objects)
print("worker 3: ", worker_3._objects)
print("worker 4: ", worker_4._objects)
print("worker 5: ", worker_5._objects)

worker 1:  {}
worker 2:  {}
worker 3:  {}
worker 4:  {}
worker 5:  {}
